### Trend, Seasonality and Residual Decomposition
EDA plotting additive or multiplcative decomposition into trend, seasonal and residual graphs to see how they compare with observed curved. eg. great variance in residual in certain time period suggest unpredicted events in those period. No pattern in seasonal means there is no seasonality.

```
from statsmodels.tsa.seasonal import seasonal_decompose

dec_add = seasonal_decompose(df.value, model = 'additive')
dec_add.plot()

dec_mult = seasonal_decompose(df.value, model = 'multiplicative')
dec_mult.plot()

```

### Stationarity
Removing Trend, Irregularity and Seasonality out of data.
Consecutive samples of the same size (time period) should have the same 
- constant mean
- constant variance
- same covariance between first period and next period

### Augmented Dickey-Fuller Test
To test for stationarity, based on null hypothesis 

$H_0 : \varphi_1 < 1$ - Non- stationary

$H_1 : \varphi_1 = 1$ - Stationary

if test statistic < critical value = stationary process

```
import statsmodels.tsa.stattools as sts
sts.adfuller(df.value)
```
**Results Example:**
```
(-1.77709238,
0.49305483054,
18,
5002,
{'1%': -3.34345
 '5%': -2.34534,
 '10%': -2.4544},
399923.2342342)

(test_statistic, p-value, number of lags, observations used in regression, critical_values, maximised information criteria lower better)
```
**Result interpretation**
- test-statistic is more than critical values
- p-value - 40% chance of accepting null, $\varphi$ is close to <1 and close to 0
- time series comes from non-stationary process


### Autoregressive model (AR model)
A linear model, where current period values $r_t$ are a sum of past outcomes $r_{t-n}$ multipled by a numeric factor $\varphi$, using a constant benchmark $C$ and including unpredictable shocks from predicted $\varepsilon$

$$
r_t = C + \varphi r_{t-1} + \varepsilon_t
$$

or

$$
r_t = C + \varphi_1 r_{t-1} + \varphi_2 r_{t-2} + ... + \varepsilon_t
$$

where 
- C is a constant benchmark, usually the first period
- $\varphi$ is a portion of the lagged variable $x_{t-n}$, a coefficient (slope) between -1 to 1
- $\varepsilon_t$ is the residual difference between our prediction for period "t" and the correct value. Basically the unpredicatable shocks

### Autocorrelation Function (ACF) 
- Computes correlation of original data and different lag versions of itself
- To determine appropriate number of lags as seen from plots
- Coefficient(correlation) $\varphi$ should be significantly above 0 (outside of blue zone) to have an impact in the AR Model
- the more lags, the significance zone gets larger, so correlation need to be higher to be significant. This is because the today's price is closer to yesterday's price, sufficiently greater to be of significance as autocorrelation seldom persists the further the distance in time.

```
import statsmodels.graphics.tsplots as sgt 
sgt.plot_acf(df.value, zero = False, lags = 40)
```

### Partial Autocorrelation Function (PACF)
- Same as ACF, but compares the direct effect of how one past lag has on the present (2 periods without the in between)

`sgt.plot_pacf(df.value, zero = False, lags = 40, alpha = 0.05, method = 'ols')`

##  Using percentage change for non-stationary data

- use Dickey-fuller test to confirm non-stationarity
- Percentage change between price of current period - price of period before

$(Price_{t} - Price_{t-1}) /  Price_{t-1} * 100$

```
df.value.pct_change(1).mul(100)

# pct_change default uses 1 time period, returns real number like 0.02
# mul(X) multiplies by 100 
```

- note that normalizing (making numbers into percentage) will not affect model or make it stationary

# AR Model

- Using PACF to identify initial optimum lags to start modelling
- use returns (percentage_change) which is stationary rather than price

$$
r_t = C + \varphi r_{t-1} + \varepsilon_t
$$

```
from statsmodels.tsa.arima_model import ARMA
```
- AR 1 model
```
model_ar = ARMA(df.value, order = (1, 0)) 

#order = (AR, MA)
#1 is the number of past values (lags) we wish to incorporate into the AR model
#0 is that we are not taking any residual values into consideration. MA model

results_ar = model_ar.fit()
results_ar.summary()

```
- AR 2 model
```
model_ar_2 = ARMA(df.value, order = (2, 0)) 
```

- compare constant coef ($C$) and AR1 coef ($\varphi$)
- p-value, if less than 0.05, coef is significantly different from 0 (reject null hypothesis that coef is 0)
- p-value, if more than 0.05, coef is not significant different from 0, which means it is close to 0, and $\varphi x_{t-1}$ has no effect on model.

- Checking between models with more lags, log-likelihood should increase, and AIC, BIC, HQIC (known as information criteria) should decrease.

- Best model need to satisfy 2 conditions
1. Significant p-value for LLR test, with the next lag having non-significant p-value for LLR test 
2. Significant p-value for lag coefficient, with the next higher lag having non-significant p-value.

- So if the last model has a significant p-value for both these conditions, the previous model would be the best model


## Log-likelihood Test
- Compare between AR models with different lags

- mod_1 should be the model with lower lag
- check for greater log-likelihoods
- a function that returns p-value, an acceptable AR model should have a non-significant p-value for LLR

```
from scipy.stats.distributions import chi2
from stats .. import llf

def LLR_test(mod_1, mod_2, DF=1):
    L1 = mod_1.fit().llf
    L2 = mod_2.fit().llf
    LR = (2*(L2-L1))
    p = chi2.sf(LR, DF).round()
    return p
    
```

## Error residuals

Should resemble white noise for a good model
-- constant variance, mean, stationarity

- Find residual of the good model
```
df['res_price] = model_ar_7.fit().resid

```
or if already know results of the model

```
df['res_price] = results_ar_7.resid

```
- check variance and mean, if high, suggest that the residuals are not concentrated around the mean. If close to 0, indicate it is like white noise (constant variance, constant mean), and our model is good

- Gaussian White noise follows normality, so we can measure up to 3 standard deviations away to define whether variance is too much. Eg. [-3.5%, 3.5%] = 7% variance is a lot for market returns

Standard deviation = sqrt(variance)

3 std deviation = 3*sqrt(variance)

```
df.res_price.var()

```
- run dicky-fuller test to check stationarity i.e. p-value <0.05

```
sts.adfuller(df.res-price)
```

- examine ACF of residual prices
if within blue zone, it is not significantly different from zero, which fits characterisitc of white noise. if outside of blue zone, suggests there are lags that can predict better.

- The further away the lags, the less relevant coefficients even if they seem signficant, this is because markets adjust to shocks, so values far in the past lose relevance.

- plot residual prices, it should look like white noise for a good model
`df.res_price[1:].plot()`



## Moving Average MA Model

To absorb all the unpredicted shocks and move the mean along.
- Main difference from AR model is the $\theta_1$ coefficient and $\varepsilon_{t-1}$ which takes into account of residual of previous period instead of past outcomes eg. price

- Uses ACF to estimate number of lags required for best model instead of PACF, because direct effect of past outcomes does not determine current outcome

# MA Model

$$
r_t = C + \theta_{1} \varepsilon_{t-1} + \varepsilon_t
$$

```
from statsmodels.tsa.arima_model import ARMA
```
- MA 1 model
```
model_ret_ma_1 = ARMA(df.value, order = (0, 1)) 

#order = (AR components, MA components)

results_ret_ma_1 = model_ret_ma_1.fit()
results_ret_ma_1.summary()
```
- MA 2 model
```
model_ret_ma_2 = ARMA(df.value, order = (0, 2)) 

results_ret_ma_2 = model_ret_ma_2.fit()
results_ret_ma_2.summary()
```
- LLR Test p-value to check if model is a signficantly better fit

- If the last model has a significant p-value >0.05 for both LLR test and lag coefficient, the previous model would be the best model

- Check ACF for lags with higher coefficient and compare with the best fit model. Example, run LLR test between MA(6) vs MA(8), changing degree of freedom to DF=2 as 8 - 6 = 2 more variables.

`LLR_test(MA_6, MA_8, DF=2)` 

- Examine Residuals as per AR process

# ARMA model

- Combine both AR and MR, allow AR models to calibrate faster and adjust to huge shocks, gives MA models foundation for predicitions than constant.

- AR components: Past values and associated coefficient $\varphi$ 
- MA components: past residuals and associated coefficient $\theta_{1}$

$$
r_t = C + \varphi r_{t-1} + \theta_{1} \varepsilon_{t-1} + \varepsilon_t
$$

- Start with an over-parameterised ARMA model
- However unable to use ACF or PACF for estimating lags, optimal ARMA model would contain fewer components of each type. EG. AR(6) and MA(8) can explain on their own we shouldn't need both lags but lesser. So we can 'half' the components and go for ARMA(3,4) or ARMA(3,3) for simplicity.

- ARMA(1,1) model
```
model_ret_ar_1_ma_1 = ARMA(df.value, order = (1, 1)) 

#order = (AR components, MA components)

results_ret_model_ar_1_ma_1 = model_ret_ar_1_ma_1.fit()
results_ret_ar_1_ma_1.summary()
```

- Compare coefficient p-value and LLR test p-value
- Make sure there is nesting between models when using LLR test, otherwise if both models have same degree of freedom eg. DF=6 for both ARMA(1, 5) and ARMA(5, 1), compare based on summary stats LLR (ideal should be higher) and AIC value (ideal should be lower)
- Nesting between 2 models ARMA($P_1, Q_1$) and nested ARMA($P_2, Q_2$) must satisfy 3 conditions
    1. $P_1 + Q_1 >= P_2 + Q_2$
    2. $P_1 >= P_2$
    3. $Q_1 >= Q_2$

- Investigate residual using ACF to identify better lags and check for possible models as ARMA cannot use ACF (MA model) or PACF (AR Model) to identify initial optimal lag

- Make sure best model has residuals are non-significant (basically random) within the first 10 lags

## ARIMA
- Adding Integration to ARMA to model non-stationary data because ARMA does not perform well with non-stationary data like prices and index
- Not advisable to use ARIMA on converted/stationary data because there is multiple integration causing more computation costs, hard to interpret results if the values are converted because differences become narrower
- Integration is like converting price into returns (pct_change) before running ARMA although this is a brute integration
- ARIMA(p,d,q)

p: AR components

d: number of times to integrate the time-series to ensure stationarity

q: MA components

eg. ARIMA(p, 0, q) = ARMA(p, q)

eg. ARIMA(0, 0, q) = MA(q)

eg. ARIMA(p, 0, 0) = AR(p)

$$
\bigtriangleup P_t = C + \varphi\bigtriangleup P_{t-1} + \theta_{1} \varepsilon_{t-1} + \varepsilon_t
$$

$\bigtriangleup P_t$ is change in value, like returns.

eg. for ARIMA(p, 2, q) there are 2 integrations, 1st integration is the change in value, 2nd integration will be the change between change in value

- For any integration we lose a single observation like 2x integration will lose 2 days of observation.

```
from statsmodels.tsa.arima_model import ARIMA

model_ar_1_i_1_ma_1 = ARIMA(df.value, order=(1,1,1))
results_ar_1_i_1_ma_1 = model_ar_1_i_1_ma_1.fit()
results_ar_1_i_1_ma_1.summary()
```
- Note that in summary, there is no coefficent for integration based on the formula. The integration order(d) has no effect on the number of parameters we need to estimate, we are only transforming underlying data to stationary 

- Check residual ACF to check for optimal lags to model. Note that ACF will fail to compute due to missing observations. so we have use the residuals from the second period(for the case of d=1) i.e. using indexing `[1:]` for d=1 or if d=2 use `[2:]`

```
df['res_ar_1_i_1_ma_1] = results_ar_1_i_1_ma_1.resid
sgt.plot_acf(df.res_ar_1_i_1_ma_1[1:], zero = False, lags =40)
```

- Comparing LLR and AIC using summary stats

```
print("ARIMA(1,1,1): \t LL = ", results_ar_1_i_1_ma_1.llf, "\t AIR = ", results_ar_1_i_1_ma_1.aic)

print("ARIMA(1,1,2): \t LL = ", results_ar_1_i_1_ma_2.llf, "\t AIR = ", results_ar_1_i_1_ma_2.aic)
```

- Run LLR test if models are nested, to rationalise the more complex models
```
print("\n LLR test p-value = " + str(LLR_test(model_ar_1_i_1_ma_1, model_ar_1_i_1_ma_3, DF=2)))

#degrees of freedom need to adjust if there is more than 1
```
- Examine ACF residual again to check if they are all insignifacnt or if there exist a better lag model


## ARIMAX

- Includes consideration of external factors "MAX" models.
- Good for analysing data as adding external factors may vastly improve modelling performance, but because forecasting requires future data of these external factors, if this is not available, it will not be able to forecast.
- ARMAX is the non-integrated version and ARIMAX is the integrated version
- Additional $\beta$ coefficient and  $X$ variable that we are interested in like 
    - time-varying measuremnet like inflation, index prices
    - categorical variable like seperating days of the week
    - boolean value like different festive periods
    - combination of other external(exogeneous) factors as long as it can affect prices and we have the data available for every period


$$
\bigtriangleup P_t = C + \beta X + \varphi\bigtriangleup P_{t-1} + \theta_{1} \varepsilon_{t-1} + \varepsilon_t
$$

$\bigtriangleup P_t$ is change in value, like returns.

- ARIMAX (1,1,1)
- Add `exog=array` argument into model, eg. if using S&P500 data we will put `exog=df.spx`

```
model_ar_1_i_1_ma_1_Xspx = ARIMA(df.value, exog = df.spx order=(1,1,1))
results_ar_1_i_1_ma_1_Xspx = model_ar_1_i_1_ma_1_Xspx.fit()
results_ar_1_i_1_ma_1_Xspx.summary()
```

## SARIMAX

- Seasonal data with periods that are longer apart eg. every christmas will be 12 months apart

- SARIMAX (p, d, q)(P, D, Q, s)

(P,D,Q): is the seasonal AR component, seasonal integration, seasonal MA component

s: Length of cycle. The number of periods needed to pass before the tendency reappears. If s=1 there is no seasonality. We set s=5 for 5 business day week, and the PQpq values should be lesser than 5 

- SARIMAX (1, 0, 2)(2, 0, 1, 5) should have P+Q+p+q = 6 coefficients

$$
y_t = C + \varphi y_{t-1} + \theta_{1} \varepsilon_{t-1} + \theta_{1} \varepsilon_{t-2} + \phi_{1} (y_{t-5} + \varphi y_{t-6}) + \phi_{2} (y_{t-10} + \varphi y_{t-11}) + \Theta_{1}(\varepsilon_{t-5} + \theta_{1} \varepsilon_{t-6} + \theta_{2} \varepsilon_{t-7}) + \varepsilon_t
$$

p : $\varphi y_{t-1}$

q : $\theta_{1} \varepsilon_{t-1}   +   \theta_{1} \varepsilon_{t-2}$

Seasonal P : $\phi_{1} (y_{t-5} + \varphi y_{t-6})   +   \phi_{2} (y_{t-10} + \varphi y_{t-11})$

Seasonal Q : $\Theta_{1}(\varepsilon_{t-5} + \theta_{1} \varepsilon_{t-6} + \theta_{2} \varepsilon_{t-7})$

```
from statsmodels.tsa.statespace.sarimax import SARIMAX
```
- SARIMAX (1,0,1) (2, 0, 1, 5) using S&P500 as exogeneous variable

```
model_sarimax = SARIMAX(df.value, exog = df.spx order=(1,0,1), seasonal_order = (2,0,1,5))
results_sarimax = model_sarimax.fit()
results_sarimax.summary()

```


## ARCH

- Autoregressive Conditional Heteroskedasticity model
- To measure volatility and exaggerate variance so we can find stability
- Measuring variance $\sigma^2$ dependent on past values' variance
- ARCH is used to predict future variance rather than future returns, to expect to see stability in the market but not predict whether the price will go up or down.

**ARCH equation**
$$
Var (r_t | r_{t-1}) = \sigma^2_t = a_0 + a_1\varepsilon^2_{t-1} + a_2\varepsilon^2_{t-2} + ... + a_q\varepsilon^2_{t-q}
$$

**ARMA - ARCH model**
- uses concept of 2 equations - mean $\mu$ equation and variance $\sigma^2$ equation
- First fits mean equation to the data and estimates residuals $\varepsilon_t$
- Then based on $\varepsilon_t$ extracted, estimates the variance

1. **mean equation** (combination of 2 equations below)
$$
r_t = C_0 + \varphi_1\mu_{t-1} + \varepsilon_t
$$
 
*mean that follows return and observes some white noise*
$$
r_t = \mu_t + \varepsilon_t
$$
*mean is a function reflective of past values and past errors with no residual as included in 1st equation*
$$
\mu_t = C_0 + \varphi_1 \mu_{t-1}
$$

2. **variance equation** to handle the shocks
$$
\sigma^2_t = a_0 + a_1\varepsilon^2_{t-1} + a_2\varepsilon^2_{t-2} + ... + a_q\varepsilon^2_{t-q}
$$

- Use PACF to observe returns and squared returns to see estimate number of lags required for model

ARCH(q) - where q is the number of previous values we include in the model
- ARCH(1)
    - set volatility vol = 'ARCH' 
    - set mean = constant if do not know if it is 0, normally assumed as time invariant. Can be set to mean = 'AR' if we know mean is an autoregressive
    - set p=1 which is order of the model like ARMA(P), number of periods to take into consideration.

```
from arch import arch_model

model_arch_1 = arch_model(df.returns[1:], mean = 'Constant', vol = 'ARCH', p=1)
results_arch_1 = model_arch_1.fit()
results_arch_1.summary()

results_arch_1 = model_arch_1.fit(update_freq=5)
to avoid seeing all iteration results
```
**Summary stats**

- Constant Mean - GARCH Model Results
    - Mean Model: Constant Mean
    - Vol Model: GARCH
    - Distribution: Normal (of the residuals)
    - DF model: number of degree of freedoms / number of coefficients which comes from $C_0$ in mean equation, $a_0$ and $a_1$ in variance equation
    - R-squared: a measurement of explanatory variation away from the mean. If value is 0, it means residuals are a version of the original data set where every value is decreased by a constant, then there will be no variance and nothing to explain

- Mean Model
    - mu is a single value because we have a constant mean model
    - p-value suggest significance of coefficient of mu, so that we know it is not close to 0

- Volatility Model
    - omega represents $a_0$ of variance equation
    - alpha[1] represents $a_1$ coefficient of squared error terms
    - beta[1]

**Results**  
- Iterations: 6 means that model is light
- Log Likelihood should increase, model will auto stop at the best model, if we don't want to see all iterations we can set argument in `fit(update_freq=5)`
- Compare Log Likelihood with other models
- Ensure all coefficients are significant
- However ARCH is used to predict future variance rather than future returns, to expect to see stability in the market but not predict whether the price will go up or down.

## GARCH

- Generalised Autoregressive Conditional Heteroskedasicity
- ARCH equation + Additional conditional variance from the last period to create a benchmark to measure volatility

$$
Var (y_t | y_{t-1}) = \Omega + a_1\varepsilon^2_{t-1} + \beta_1 \sigma^2_{t-1}
$$
- Because additional variance contains same residual in the ARCH equation, only GARCH(1,1) will deliver results while higher orders of more conditional variance will be redundant.
- Adding a single past variance gives more predictive power than 11 squared residuals eg. GARCH(1,1) vs ARCH(12) when comparing log-likelihood

- GARCH(1,1)
```
from arch import arch_model

model_garch_1_1 = arch_model(df.returns[1:], mean = 'Constant', vol = 'GARCH', p=1, q=1)
results_garch_1_1 = model_garch_1_1.fit()
results_garch_1_1.summary()
```